In [19]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [148]:
with open('d:\__ HKEX __ HKEXnews __.html','r',encoding="utf-8") as f:
    html_txt = f.read()

In [9]:
soup = BeautifulSoup(html_txt,'html.parser')

In [25]:
temp_html = soup.find("table",{"id":"Table5"})
temp_html = temp_html.find_all('td',{"class":"arial12black"})

first_dict = {}
first_dict['last_date'] = temp_html[1].get_text().strip()
first_dict['stock_code'] = temp_html[3].get_text().strip()
first_dict['stock_name'] = temp_html[5].get_text().strip()

print(first_dict)

{'last_date': '13/06/2018', 'stock_code': '91888', 'stock_name': '中國國旅'}


In [26]:
temp_html = soup.find("div",{"id":{"pnlResultSummary"}})
temp_html = temp_html.find_all("span",{"class":"mobilezoom"})

second_dict = {}
second_dict['hold_volumn'] = temp_html[0].get_text().strip()
second_dict['people_number'] = temp_html[1].get_text().strip()
second_dict['hold_precent'] = temp_html[2].get_text().strip()
second_dict['all_volumn'] = temp_html[6].get_text().strip()
print(second_dict)

{'hold_volumn': '215,446,668', 'people_number': '52', 'hold_precent': '11.03%', 'all_volumn': '1,952,475,544'}


In [22]:
temp_html = soup.find("table",{"id":{"participantShareholdingList"}})
temp_html = temp_html.find_all('tr')

row_number = len(temp_html)
col_number = len(temp_html[0].find_all('td'))

d = {}
for i in range(col_number):
    d[i] = []

for i in range(1,row_number):
    temp_tr = temp_html[i]
    temp_td = temp_tr.find_all('td')
    
    if len(temp_td) != col_number:
        continue
        
    for j in range(col_number):
        temp_string = temp_td[j].get_text().strip()
        d[j].append(temp_string)

last_pd = pd.DataFrame(d)
    
last_dict = {}
for i in range(len(last_pd)):
    last_dict[last_pd.iat[i,0]] = last_pd.iat[i,1]

new_pd = pd.DataFrame(last_pd[3])
new_pd = new_pd.T
new_pd.columns = last_pd[0]

In [24]:
new_pd

,C00019,C00039,C00010,C00100,B01161,B01110,C00093,B01565,B01654,B01491,...,B01372,B01971,B01695,B01825,C00036,B01947,B01353,C00042,B01818,B01967
3,"57,653,019","36,854,187","28,569,841","26,046,799","12,900,300","10,594,931","9,752,274","8,684,492","6,362,971","5,401,751",...,"5,000","3,200","2,000","2,000","1,800","1,700","1,100","1,000",400,100


In [81]:
new_pd.index = {first_dict['last_date']}

In [84]:
merge_pd = new_pd

In [ ]:
merge_pd = pd.concat([old_pd,new_pd])
name_dict.update(last_dict)

store['name_dict'] = name_dict
store['old_pd'] = merge_pd


In [28]:
import openpyxl as oxl

In [37]:
wb = oxl.Workbook()

In [38]:
ws = wb.create_sheet(index=0,title='oxl-sheet')

In [39]:
ws.cell(row=1,column=1).value = '持股日期'
ws.cell(row=1,column=2).value = first_dict['last_date']

ws.cell(row=1,column=3).value = '股票代码'
ws.cell(row=1,column=4).value = first_dict['stock_code']

ws.cell(row=1,column=5).value = '股票名称'
ws.cell(row=1,column=6).value = first_dict['stock_name']

In [43]:
ws.cell(row=3,column=1).value = '中央结算系统持股量'
ws.cell(row=4,column=1).value = second_dict['hold_volumn']

ws.cell(row=3,column=2).value = '参与者数目'
ws.cell(row=4,column=2).value = second_dict['people_number']

ws.cell(row=3,column=3).value = '总数百分比'
ws.cell(row=4,column=3).value = second_dict['hold_precent']

ws.cell(row=3,column=4).value = '全部持股量'
ws.cell(row=4,column=4).value = second_dict['all_volumn']

In [54]:
columns = merge_pd.columns
columns_name = []

for i in range(len(columns)):
    columns_name.append(last_dict[columns[i]])

In [66]:
for i in range(len(columns)):
    ws.cell(row=7,column=i+3).value = columns_name[i]

In [67]:
ws.cell(row=8,column=1).value = '日期'
ws.cell(row=8,column=2).value = '收盘价'

In [87]:
index = merge_pd.index

In [62]:
row_number = len(index)
col_number = len(columns)

In [145]:
for i in range(row_number):
    ws.cell(row=i+9,column=1).value = index[i]
    ws.cell(row=i+9,column=2).value = close_data[index[i]]

In [65]:
for i in range(col_number):
    ws.cell(row=8,column=i+3).value = columns[i]

In [78]:
for i in range(row_number):
    for j in range(col_number):
        ws.cell(row=9+i,column=j+3).value = merge_pd.iat[i,j]

In [71]:
import tushare as ts

In [144]:
close_data = OrderedDict()
for i in range(row_number):
    temp_s = index[i]
    temp_str = datetime.datetime.strptime(temp_s,'%d/%m/%Y').date().isoformat()
    
    if not temp_s in close_data:
        temp_df = ts.get_k_data('601888',start=temp_str,end=temp_str,ktype='D')
        if temp_df.empty:
            close_data[temp_s] = list(close_data.values())[-1]
        else:
            close_data[temp_s] = temp_df.iat[-1,2]
    

In [ ]:
for i in range(row_number):
    ws.cell(row=i+9,column=2).value = close_data[i]

In [103]:
temp_str = index[i]

In [147]:
from collections import OrderedDict

In [142]:
close_data

OrderedDict([('2018-06-13', 66.819999999999993)])

In [93]:
import datetime

In [104]:
temp_str = datetime.datetime.strptime(temp_str,'%d/%m/%Y').date().isoformat()

In [121]:
temp_df = ts.get_k_data('601888',start=temp_str,end=temp_str,ktype='D')

In [128]:
close_data[temp_str] = temp_df.iat[-1,2]

In [139]:
list(close_data.values())[-1]

66.819999999999993

In [109]:
temp_df.iat[-1,2]

66.819999999999993

In [146]:
wb.save('test.xlsx')